In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import os, glob, re, sys, random, unicodedata, collections
from tqdm import tqdm
from functools import reduce
import nltk
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
%matplotlib inline

In [75]:
a=pd.read_json("meta-data.jsonl",lines=True)

In [89]:
a=a.loc[:,["ABSTRACT","TITLE","LANGUAGE","DBRECORDID"]]

In [96]:
a=a[(a["ABSTRACT"].notnull())]

In [98]:
a.index=range(0,len(a))

In [5]:
c=pd.read_json("livivo_hq_100_candidates.jsonl",lines=True)

In [6]:
c

,qid:,qstr,candidates
0,1,covid-19 OR sars-cov-2,"[M33634079, M32328927, M32467201, M32333579, M..."
1,2,dementia,"[M32016328, M32016329, M30925608, M31476229, M..."
2,3,biodiversity,"[M31517113, M28569800, M32821644, M32295086, M..."
3,4,influenza,"[M32048225, M31534258, M30722953, M31478058, M..."
4,5,malaria,"[M30824502, M30550661, M29813596, M29813666, M..."
...,...,...,...
995,1446,epigenetik,"[M30643952, M29632998, M27299943, M29560499, M..."
996,1452,stroke AND unit,"[M33389780, M33394911, M33013663, M32115340, M..."
997,1453,praxis AND der AND intensivmedizin,"[M29556682, M22441683, M33053589, M32086542, M..."
998,1454,das AND gesundheitssystem AND in AND deutschland,"[M31594009, M31396676, M32023648, M31590198, M..."


## Pre Processing

In [100]:
#Datas are stored as a list in every cell, this operation applied to extract the list
for i in a.columns:
    a[i]=a[i].apply(lambda x: x[0] if isinstance(x, list) else x)

In [101]:
a["LANGUAGE"].value_counts()

eng               85375
ger               35297
lat                1212
spa                 968
por                 689
                  ...  
german                1
Serbo-Croatian        1
GER                   1
English|German        1
en-US                 1
Name: LANGUAGE, Length: 72, dtype: int64

In [102]:
def strip_accents(text):   
    nfkd = unicodedata.normalize('NFKD', text)
    newText = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z0-9 \\\']', ' ', newText)

def tokenize_text(text):
    """Make all necessary preprocessing of text: strip accents and punctuation, remove the words only contains digit
    remove \n, tokenize our text, convert to lower case, remove stop words and 
    words with less than 2 chars.

    Parameters:
    text (str): Input text

    Returns:
    str: cleaned tokenized text

   """    
    WORD_MIN_LENGTH = 2
    STOP_WORDS = nltk.corpus.stopwords.words('english')
    text = strip_accents(text)
    text = re.sub(re.compile('\n'),' ',text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in STOP_WORDS and len(word) >= WORD_MIN_LENGTH]
    words = [word for word in words if word.isdigit()==False]
    return words

In [103]:
a["ABSTRACT"]=a["ABSTRACT"].apply(lambda x: strip_accents(x))

In [104]:
tokenized=(a["ABSTRACT"].apply(lambda x: tokenize_text(x)))

## Building Inverted Index

In [105]:
def inverted_index(words):
    """Create a inverted index of words (tokens or terms) from a list of terms

    Parameters:
    words (list of str): tokenized document text

    Returns:
    Inverted index of document (dict)

   """       
    inverted = {}
    for index, word in enumerate(words):
        locations = inverted.setdefault(word, [])
        locations.append(index)
    return inverted

def inverted_index_add(inverted, doc_id, doc_index):
    """Insert document id into Inverted Index

    Parameters:
    inverted (dict): Inverted Index
    doc_id (int): Id of document been added
    doc_index (dict): Inverted Index of a specific document.

    Returns:
    Inverted index of document (dict)

   """        
    for word in doc_index.keys():
        locations = doc_index[word]
        indices = inverted.setdefault(word, {})
        indices[doc_id] = locations
    return inverted

In [106]:
inverted_index(tokenized[3])

{'ukolem': [0],
 'teto': [1],
 'prace': [2],
 'bylo': [3],
 'sestaveni': [4],
 'jednoduche': [5],
 'testove': [6],
 'baterie': [7],
 'ktera': [8],
 'citlive': [9],
 'registrovala': [10],
 'zmeny': [11, 68],
 'motoriky': [12],
 'pareticke': [13],
 'ruky': [14],
 'postizene': [15],
 'horni': [16],
 'koncetiny': [17],
 'pacientu': [18],
 'po': [19, 50],
 'cevni': [20],
 'mozkove': [21],
 'prihode': [22],
 'chronickem': [23],
 'stadiu': [24],
 'vychazeli': [25],
 'jsme': [26, 45],
 'bezne': [27],
 'dostupnych': [28],
 'testu': [29, 56],
 'nine': [30],
 'hole': [31],
 'peg': [32],
 'test': [33, 118],
 'dynamometrie': [34],
 'apod': [35],
 'ktere': [36],
 'jsou': [37],
 'pro': [38],
 'tyto': [39, 67],
 'ucely': [40],
 'praxi': [41],
 'nejvice': [42],
 'vyuzivany': [43],
 'sledovali': [44],
 'charakter': [46],
 'dynamiku': [47],
 'jejich': [48],
 'zmen': [49],
 'rehabilitacnich': [51],
 'procedurach': [52],
 'vetsina': [53],
 'vysledku': [54],
 'vybranych': [55],
 'mela': [57],
 'obdobnou': [

In [107]:
inverted_doc_indexes = {}
files_with_index = []
files_with_tokens = {}
for i in tokenized.index:
    #Clean and Tokenize text of each document
    words = tokenized[i]
    #Store tokens
    files_with_tokens[i] = words

    doc_index = inverted_index(words)
    inverted_index_add(inverted_doc_indexes, i, doc_index)
    files_with_index.append(i)

## Running Boolean Search

In [108]:
def boolean_search(inverted, file_names, query):
    """Run a boolean search with AND operator between terms over 
    the inverted index.

    Parameters:
    inverted (dict): Inverted Index
    file_names (list): List with names of files (books)
    query (txt): Query text

    Returns:
    Names of books that matchs the query.

   """      
    # preprocess the user query using same function used to build Inverted Index
    words = [word for _, word in enumerate(tokenize_text(query)) if word in inverted]
    # list with a disctinct document match for each term from query
    results = [set(inverted[word].keys()) for word in words]
    # AND operator. Replace & for | to modify to OR behavior.
    docs = reduce(lambda x, y: x & y, results) if results else []
    return ([file_names[doc] for doc in docs])

In [109]:
files_with_index= pd.DataFrame(files_with_index)
files_with_index.index=files_with_index[0]

In [110]:
boolean_search(inverted_doc_indexes, files_with_index[0] , c["qstr"][15])

[95362,
 35075,
 76420,
 120962,
 122115,
 141,
 107279,
 5396,
 81556,
 93461,
 58395,
 22813,
 107813,
 50346,
 49328,
 38195,
 47161,
 124478,
 107583,
 91458,
 97862,
 109510,
 58954,
 22091,
 7501,
 99663,
 43858,
 121181,
 34400,
 27491,
 6511,
 9840,
 44278,
 16764]

## Ranking with tf-idf

In [111]:
DF = {}
for word in inverted_doc_indexes.keys():
    DF[word] = len ([doc for doc in inverted_doc_indexes[word]])

total_vocab_size = len(DF)
print(total_vocab_size)

477456


In [172]:
tf_idf = {} # Our data structure to store Tf-Idf weights

N = len(files_with_tokens)

for doc_id in tokenized.index:
    tokens= tokenized[doc_id]
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        
        # Calculate Tf
        tf = counter[token] # Counter returns a tuple with each terms counts
        tf = 1+np.log(tf)
        
        # Calculate Idf
        if token in DF:
            df = DF[token]
        else:
            df = 0
        idf = np.log((N+1)/(df+1))
        
        # Calculate Tf-idf        
        tf_idf[doc_id, token] = tf*idf

## Ranked Search

In [173]:
def ranked_search(k, tf_idf_index, file_names, query):
    """Run ranked query search using tf-idf model.

    Parameters:
    k (int): number of results to return
    tf_idf_index (dict): Data Structure storing Tf-Idf weights to each 
                        pair of (term,doc_id) 
    file_names (list): List with names of files (books)
    query (txt): Query text

    Returns:
    Top-k names of books that matchs the query.

   """   
    tokens = tokenize_text(query)
    query_weights = {}
    for doc_id, token in tf_idf:
        if token in tokens:
            query_weights[doc_id] = query_weights.get(doc_id, 0) + tf_idf_index[doc_id, token]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    results = []
    for i in query_weights[:k]:
        results.append(file_names[i[0]])
    
    return results

In [ ]:
ranked_search(10,tf_idf,files_with_index[0], c["qstr"][1])

In [ ]:
searched= ranked_search(100,tf_idf,files_with_index[0], c["qstr"][1])

In [ ]:
result=[]
for i in searched:
    result.append(a[a.index==i]["DBRECORDID"].values[0])

In [ ]:
((pd.DataFrame(result))==(pd.DataFrame(c[c["qid:"]==1]["candidates"].values[0]))).sum()

In [ ]:
c

In [ ]:
a[a["DBRECORDID"]=="M33634079"]

In [ ]:
a[a.index==166241]["ABSTRACT"].values

## Vectorization by query, eucledian normalization and ranking by cosine similarity

#### Vectorize

In [30]:
def vectorize(q,tf_idf):
    tokens = tokenize_text(q)
    liste=[]
    for i in list(tf_idf.keys()):
        liste.append(i[0])
    liste=set(liste)
    liste = sorted(liste, key=lambda x: x, reverse=False)
    query_weights_vector={}
    for z in liste:
        for token in tokens:
            counter=0
            if token in tokenized[z]:
                counter += 1
                if counter<=1:
                    query_weights_vector[z]=[]
                    for token in tokens:
                        if token in tokenized[z]:
                            query_weights_vector[z].append(tf_idf[z,token])
                        else:
                            query_weights_vector[z].append(0)
    return query_weights_vector, tokens, ("irish acute hospital service cure medicine query")
query_weights_vector, tokens, query = vectorize("irish acute hospital service cure medicine query", tf_idf)

#### Eucledian Normalization

In [31]:
normalize = pd.DataFrame(query_weights_vector, index=tokens).T
def normalizer(x):
    y=x.apply(lambda x: x**2)
    y=y.apply(lambda x: x.sum(),axis=1)
    y=np.sqrt(y)
    y=1/y
    return normalize.apply(lambda x: x*y)
normalized=normalizer(normalize);normalized

,irish,acute,hospital,service,cure,medicine,query
3,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0
9,0.0,0.749977,0.661464,0.0,0.0,0.0,0.0
48,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0
57,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0
80,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
239687,0.0,0.626819,0.779165,0.0,0.0,0.0,0.0
239702,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0
239777,0.0,0.771608,0.636098,0.0,0.0,0.0,0.0
239782,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0


#### Cosine Ranking

In [32]:
def cosine_ranking(query_tokens, normalized, n):
    v1 = np.array([1]*len(query_tokens))
    output={}
    for i in normalized.index:
        v2 = np.array(normalized.loc[i,:].values)
        output[i]= np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))
    output = sorted(output.items(), key=lambda x: x[1], reverse=True)
    liste=[]
    for i in output:
        liste.append(i[0])
    return liste[:n]

In [33]:
cosine_ranking(tokens, normalize, 10)

[126152, 65721, 87353, 140993, 58900, 133279, 144077, 41374, 94327, 140988]

In [34]:
ranked_search(10,tf_idf,files_with_index[0], query)

[166241, 65721, 184093, 197817, 39925, 126152, 56439, 144077, 13910, 860]

#### Let's see the queries with no match

In [35]:
rankings={}
count=0
for i in c.index:
    rankings[(c.loc[i,"qid:"])]=ranked_search(10,tf_idf,files_with_index[0],c.loc[:,"qstr"][i])
    count+=1
    print("{} / {}".format(count,len(c.loc[:,"qid:"])),end='\r')

KeyboardInterrupt: 

In [ ]:
rnksrtd = sorted(rankings.items(), key=lambda x: len(x[1]), reverse=False)
def no_results(rnksrtd):
    liste = []
    for i in rnksrtd:
        if len(i[1])==0:
            liste.append(i[0])
    return liste
no_results(rnksrtd)
c.loc[c['qid:'].isin(no_results(rnksrtd))]

##### %40 of the queries had 0 match

In [ ]:
relevant_list=[]
sayac=0
for i in c["candidates"]:
    sayac+=1
    print("{}/{}".format(sayac, len(c)),end='\r')
    for z in i:
        if z not in relevant_list:
            relevant_list.append(z)

In [ ]:
len(relevant_list)

#### Sum of relevant documents for the query list is 90668

In [ ]:
sayac=0
sayac2=0
for i in a.DBRECORDID:
    sayac2+=1
    print("{}/{}".format(sayac2, len(a)),end='\r')
    if i in relevant_list:
        sayac+=1

#### We only have 48041 document in the subset

In [ ]:
sayac

### Only 3227 out of 90668 exist in our subset

## BM25


In [163]:
from rank_bm25 import BM25Okapi

In [164]:
bm25 = BM25Okapi(tokenized)

In [165]:
query= c["qstr"][0]

In [166]:
doc_scores= bm25.get_scores(tokenized_query)

In [169]:
def bm25_result(query,n):
    tokenized_query = tokenize_text(query)
    result_list = bm25.get_top_n(tokenized_query, tokenized, n=10)
    keys=[]
    for i in result_list:
        key= [k for k,v in tokenized.items() if v==i]
        keys.append(key[0])
        keys=list(set(keys))
    return keys

In [176]:
bm25_result(query,20)

[130531, 24196, 26566, 13872, 131765, 15702, 105978, 64155]

In [177]:
ranked_search(20,tf_idf,files_with_index[0], query)

[5475,
 18074,
 128853,
 85469,
 96317,
 14018,
 20161,
 17424,
 11380,
 28460,
 121553,
 81159,
 94482,
 117676,
 77975,
 116030,
 64155,
 41297,
 27755,
 64380]

In [159]:
a.loc[keys]

,ABSTRACT,TITLE,LANGUAGE,DBRECORDID
130531,Relevance Laboratory diagnosis of the novel c...,Detectability of the novel coronavirus (SARS-C...,NaN,COVID19::1e746e81c39fc21f50a1f9fe7ca1e39fe
24196,Retest Positive for severe acute respiratory...,"Retest positive for SARS-CoV-2 RNA of ""recover...",eng,M32492212
26566,Purpose of review Severe acute respiratory s...,Lethal zoonotic coronavirus infections of huma...,eng,M33660619
13872,Background Laboratory diagnosis of the novel ...,Detectability of the novel coronavirus (SARS-C...,NaN,COVID19::12632948535e57ddd33f6c84513eeef10
131765,Facing the ongoing pandemic caused by SARS CoV...,Unexpected diagnosis of COVID-19-associated di...,eng,M32890937
15702,Zoonotic coronavirus disease COVID has emerg...,"The emerging SARS-CoV, MERS-CoV, and SARS-CoV-...",eng,M33642804
105978,Relevance Laboratory diagnosis of the novel c...,Detectability of the novel coronavirus (SARS-C...,rus,BASE::ftarxivpreprints:oai:arXiv.org:2009.02962
64155,Coronavirus disease 2019 COVID 19 pandemic...,Molecular characterization and amino acid homo...,NaN,COVID19::110a169a630251f436b623abbec060c19


In [160]:
keys

[130531, 24196, 26566, 13872, 131765, 15702, 105978, 64155]